<a href="https://colab.research.google.com/github/sanchit2843/ds_competitions/blob/master/toxicity_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:


# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
# #print(os.listdir("/kaggle/input/test"))

import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
from torchsummary import summary
import torch.optim as optim
import copy
import os
import torch
from tqdm.autonotebook import tqdm
from torch.optim.lr_scheduler import _LRScheduler
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
# Any results you write to the current directory are saved as output.

from tqdm import tqdm

tqdm.pandas()

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [2]:


!pip install -i https://test.pypi.org/simple/ supportlib
import supportlib.gettingdata as getdata
getdata.kaggle()



Looking in indexes: https://test.pypi.org/simple/


In [0]:
## adding data and unzipping it
# !kaggle competitions download -c jigsaw-unintended-bias-in-toxicity-classification
# !kaggle datasets download -d chriscc/pickled-word-embedding
# !kaggle datasets download -d authman/pickled-glove840b300d-for-10sec-loading
# # !ls
# getdata.zipextract('/content/pickled-glove840b300d-for-10sec-loading.zip')
# getdata.zipextract('/content/pickled-word-embedding.zip')


# getdata.zipextract('/content/test.csv.zip')
# getdata.zipextract('/content/train.csv.zip')

# !ls

In [4]:
## for importing data
train = pd.read_csv("../content/train.csv")
test = pd.read_csv("../content/test.csv")
print("Train shape : ",train.shape)
print("Test shape : ",test.shape)

Train shape :  (1804874, 45)
Test shape :  (97320, 2)


In [5]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab
"""
def build_vocab(sentences, verbose = True): 
from collections import defaultdict

vocab = defaultdict(lambda : 0)
for sentence in tqdm(sentences, disable = (not verbose)):
for word in sentence: 
vocab[word] += 1
return vocab
"""



# sentences = train["comment_text"].progress_apply(lambda x: x.split()).values
# vocab = build_vocab(sentences)
# print({k : vocab[k] for k in list(vocab)[:10]})

'\ndef build_vocab(sentences, verbose = True): \nfrom collections import defaultdict\n\nvocab = defaultdict(lambda : 0)\nfor sentence in tqdm(sentences, disable = (not verbose)):\nfor word in sentence: \nvocab[word] += 1\nreturn vocab\n'

In [0]:
# set embedding layer path :

# crawl_path = '..../content/crawl-300d-2M.pkl'
glove_twitter_path = '../content/glove.twitter.27B.200d.pkl'
# glove_840_path = '..../content/glove.840B.300d.pkl'

In [0]:

"""
we have imported pickle file so. 

Of course we also need to adjust the load_embeddings function, to now handle the pickled dict.


"""
## if we use .bin form of embedding
# from gensim.models import KeyedVectors

# embeddings_index = KeyedVectors.load_word2vec_format(glove_twitter_path,binary = True)

import pickle

NUM_MODELS = 2
LSTM_UNITS = 128
DENSE_HIDDEN_UNITS = 4 * LSTM_UNITS
MAX_LEN = 220

def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float32')


def load_embeddings(path):
    with open(path,'rb') as f:
        emb_arr = pickle.load(f)
    return emb_arr

def build_matrix(word_index, path):
    embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(word_index) + 1, 300))
    unknown_words = []
    
    for word, i in word_index.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

class SpatialDropout(nn.Dropout2d):
    def forward(self, x):
        x = x.unsqueeze(2)    # (N, T, 1, K)
        x = x.permute(0, 3, 2, 1)  # (N, K, 1, T)
        x = super(SpatialDropout, self).forward(x)  # (N, K, 1, T), some features are masked
        x = x.permute(0, 3, 2, 1)  # (N, T, 1, K)
        x = x.squeeze(2)  # (N, T, K)
        return x

def train_model(learn,test,output_dim,lr=0.001,
                batch_size=512, n_epochs=4,
                enable_checkpoint_ensemble=True):
    
    all_test_preds = []
    checkpoint_weights = [2 ** epoch for epoch in range(n_epochs)]
    test_loader = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=False)
    n = len(learn.data.train_dl)
    phases = [(TrainingPhase(n).schedule_hp('lr', lr * (0.6**(i)))) for i in range(n_epochs)]
    sched = GeneralScheduler(learn, phases)
    learn.callbacks.append(sched)
    for epoch in range(n_epochs):
        learn.fit(1)
        test_preds = np.zeros((len(test), output_dim))    
        for i, x_batch in enumerate(test_loader):
            X = x_batch[0].cuda()
            y_pred = sigmoid(learn.model(X).detach().cpu().numpy())
            test_preds[i * batch_size:(i+1) * batch_size, :] = y_pred

        all_test_preds.append(test_preds)


    if enable_checkpoint_ensemble:
        test_preds = np.average(all_test_preds, weights=checkpoint_weights, axis=0)    
    else:
        test_preds = all_test_preds[-1]
        
    return test_preds



In [0]:
# ## TODO:
# ## find and remove sprecial characters 


## Don't run this


# def bad_preprocess(data):
#     '''
#     Credit goes to https://www.kaggle.com/gpreda/jigsaw-fast-compact-solution
#     '''
#     punct = "/-'?!.,#$%\'()*+-/:;<=>@[\\]^_`{|}~`" + '""“”’' + '∞θ÷α•à−β∅³π‘₹´°£€\×™√²—–&'
#     def clean_special_chars(text, punct):
#         for p in punct:
#             text = text.replace(p, ' ')
#         return text

#     data = data.astype(str).apply(lambda x: clean_special_chars(x, punct))
#     return data



In [0]:
import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    covered_word_count  = 0
    oov_word_count  = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            covered_word_count += vocab[word]
        except:

            oov[word] = vocab[word]
            oov_word_count  += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(covered_word_count / (covered_word_count + oov_word_count )))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

# Side Note:

# It seems that k += vocab[word] will only be run if word can be found in embeddings_index,
# otherwise i += vocab[word] will be run. So they are not the same. 
# You can actually replace try-except with an if-else.

# Also, vocab seem to store the frequency of a word in the corpus,
# so I think k and i represent the number of words covered in the corpus.

In [10]:
# oov = check_coverage(vocab,embeddings_index)
import time
tic = time.time()
## lead and take note here


###
glove_embeddings = load_embeddings(glove_twitter_path)
# loaded 1193514 word vectors in 7.11469292640686s
# Found embeddings for 4.22% of vocab
# Found embeddings for  74.85% of all text


print(f'loaded {len(glove_embeddings)} word vectors in {time.time()-tic}s')

loaded 1193514 word vectors in 6.848312854766846s


In [11]:
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
oov[:10]

100%|██████████| 1670966/1670966 [00:03<00:00, 529971.59it/s]


Found embeddings for 4.22% of vocab
Found embeddings for  74.85% of all text


[('I', 861783),
 ('The', 435047),
 ("don't", 178881),
 ('Trump', 156956),
 ('It', 153815),
 ('You', 144381),
 ('If', 143987),
 ('And', 128132),
 ('This', 121363),
 ("it's", 100959)]

In [13]:
import string
latin_similar = "’'‘ÆÐƎƏƐƔĲŊŒẞÞǷȜæðǝəɛɣĳŋœĸſßþƿȝĄƁÇĐƊĘĦĮƘŁØƠŞȘŢȚŦŲƯY̨Ƴąɓçđɗęħįƙłøơşșţțŧųưy̨ƴÁÀÂÄǍĂĀÃÅǺĄÆǼǢƁĆĊĈČÇĎḌĐƊÐÉÈĖÊËĚĔĒĘẸƎƏƐĠĜǦĞĢƔáàâäǎăāãåǻąæǽǣɓćċĉčçďḍđɗðéèėêëěĕēęẹǝəɛġĝǧğģɣĤḤĦIÍÌİÎÏǏĬĪĨĮỊĲĴĶƘĹĻŁĽĿʼNŃN̈ŇÑŅŊÓÒÔÖǑŎŌÕŐỌØǾƠŒĥḥħıíìiîïǐĭīĩįịĳĵķƙĸĺļłľŀŉńn̈ňñņŋóòôöǒŏōõőọøǿơœŔŘŖŚŜŠŞȘṢẞŤŢṬŦÞÚÙÛÜǓŬŪŨŰŮŲỤƯẂẀŴẄǷÝỲŶŸȲỸƳŹŻŽẒŕřŗſśŝšşșṣßťţṭŧþúùûüǔŭūũűůųụưẃẁŵẅƿýỳŷÿȳỹƴźżžẓ"
white_list = string.ascii_letters + string.digits + latin_similar + ' '
white_list += "'"


glove_chars = ''.join([c for c in tqdm(glove_embeddings) if len(c) == 1])
glove_symbols = ''.join([c for c in glove_chars if not c in white_list])
glove_symbols

100%|██████████| 1193514/1193514 [00:00<00:00, 2241347.75it/s]


'.:,!"?()！。-、…/*>^？<・&♥“”_♡´،~;（･|[）]—笑ω❤～★`$و♪=｀¿+☆☺ﾟ▽%✔@؟•█｡в\\；¡＾／→∀：＼°и←⌣#я}{░д✌˘¬━–сﾉه⭕»☀あ☹＿月✅➊＞«⌒＜●∇а➌➋え♦➍♫─○ノ✋‾▀日▄؛═，➎́๑┓↘✨✈⇒┃ㅋу̀ヽ∩×≦≧ーب↙‹┌ε▒＆＠☝║☯―σ┛▿円ｗ┐£о›｣≡んي┗位ƪ▓┏｢☑►ʃ┈فⓜ╗↓っم❄♬年̷ع時╯三＝╰､※╔·は■％｜◎ˆ∠＊☞⊂✰к아お人◦╮◠€̮‿▂з╚➡と╭⊃╝ゝ◡˙º□❌└©↑☜で⊙．艸¯◇╩̩➏¼＃╦ヾ☻∧▼－回ㅠ◆╱✿いۈ이➒◔♻✧◞그＋◉˚∂일수╬╲のつ̯◟╹لｏ⛄더ิ난┊第が分²▬➐✊❗다왜내⁰∞나ا￥┻なね⁾ง®ˇ☁ⓔ泣¸┘╥ゞ͡╠ﷺ✓①˛¨−잘월か⚪⚡②◕ｰ┳щ۳私う좀［を］▇ﻻ☔▸╣³⚽̶⭐̅՞ま♩✳今❀౪◄ⓣฺु△ۆ∵♂한◾۶▅√θ†▔◯すⓞ✖⚾′俺™土것੭안☼❛③ηツ▲❁℃ㅎ✩저시ο헐또전➜♔┣◝٩にｔ彡▶歳◜金▐년や✗♛へᴗ♣◍☕だ▕ª͟し⚫¥❥네わ¤제‸뭐ㅇⓡ⇦╋ヮ本할ふ│④중❶пこ̥ⓦ♠ⓛ¦▾❷そ❸た☎‵▉ロ枚┫✴☉卍سも꼭さ분ฅг水⇢⁽ﷲ▌때ㅁ▏⇠よ✪응٣ξ❹음ك┼ρ♉와話◢نㅅدｉ❒☂汗ｪ爆☛참て₍٢∪┎⇄١의◤خू너ｖॢ⚠♕ً♏날٥１рェ가ๆ❕거м≫ザ⇨ـ§는木⌓ﺑ어건„✉⠀嵐↗ッ✽ɔ에위ㅂ火ⓕღ☃ຶ☐✂⑤͈를ʔਊ∴을◒く하るㅜ♈막∑⋆₎◀오후男◣♀┒≪３点万게▷명丿ψㅤ涙２▪е못ت̫ミ٤ืأ번ლⓨき中ν約女ⓐらжれ⛅해ةх등말ア̵원̸┉❣ム♯̃◐➫개।◥号ัンむ№∫⑥대０✦бص½イ±✡ﻭ倍장僕≖個φｑ何ⓟ巻κ۞づみ˖͒ﾝʘقι⊖ⓘ▆นぁ₃넘ﻟ夜α΄♚ち넌⬅고큰ٰ⬇야◑ʕ¢ど嘘り⇩ⓤ♧壁ر″ﻵ✏仮๏걸로н≈̴될ِ∈件☇두̾黒만มｙぇ자ｂ猫ถｐ은♍‚⑅ٍ度秒ё٪❓照첫줄حཀﾍ部∗➙全✹걍ⓒﾛ♌▃ヘａ☣求朝ԅ祝⌯☮母➬헉高☾☋赤앗⇆т곧뭘↖皿ㅡ본유名살딱ラほ회밤譲♨і二δ∋̣лَζ番◼め工他♓جｃ̪愛ڡワ및듯フч볼␞エ＂눈➑차새ひ̐⛳じメش집☚✮✘ⓑｕ一ス恋◻↔ﾞ弾된٧後٦昼ⓚ♋暇\u06dd⬛⇓⇛４⚓늘禁인도총돈５세誰代神✍ⓗ✞ご┬올怒ク님せҧ⬆ⓢวｍ¶＇니青형÷凸널̿けณハレй☄⛔ۚℓオ짱✯色小ⓝ으π➰٠ํべ妹ゆ➪╙⑦⏰온昔ドэ新著กິル夢ﾐ父\x80◓ｱอ無犬٨✝リ❖棒۔死ﯙ✫↪ろﻓ∥❝ھ̆ⓙآ┴大μ글ｘ화楽ُː➔╖몇̲白▁ㄷஇ⁼⌑✤勝ʅｴ✲ｧ☪

. We printed all symbols that we have an embedding vector for. Intrestingly its not only a vast amount of punctuation but also emojis and other symbols. Especially when doing sentiment analysis emojis and other symbols carrying sentiments should not be deleted! What we can delete are symbols we have no embeddings for. So lets check the characters in our texts and find those to delete:


In [15]:
jigsaw_chars = build_vocab(list(train["comment_text"]))
jigsaw_symbols = ''.join([c for c in jigsaw_chars if not c in white_list])
jigsaw_symbols


### we basically just deleted the symbols we have no embedding for

100%|██████████| 1804874/1804874 [00:59<00:00, 30144.28it/s]


'.,?!-;*"…:\n—()%#$&_/@＼・ω+🍕=”“[]^–>\r🐵\\°<😑~\xa0\ue014•≠\t™\uf818\uf04a\xadˈʊɒ😢🐶∞§{}·τα❤️☺ɡ\uf0e0😜😎👊\u200b\u200e😁|عدويهصقأناخلىبمغر😍💖¢→̶`💵❥━┣┫Е┗Ｏ►★👎😀😂\u202a\u202c🔥😄©―🏻💥ᴍʏʀɪᴇɴᴅᴏᴀᴋʜᴜʟᴛᴄᴘʙғᴊᴡɢ✔®\x96\x92●😋👏שלוםבי😱‼£\x81♥エンジ故障➤´\u2009🚌ᴵ͞🌟😊😳😧🙀😐😕\u200f👍😮😃😘¹☕≈÷אעכח♡◐║▬💩′ɔː💯⛽€🚄🏼ஜ۩۞†😖ᴠ🚲‐μ✒➥😟😈═☆ˌ💪🙏🎯◄🌹😇💔½ʻ😡\x7f👌ἐπὶδηλήσειὲκἀίῃἴρξνʃ🙄✬ＳＵＰＥＲＨＩＴ😠\ufeff☻±\u2028😉😤⛺♍🙂µ\u3000تحكسة👮💙فزط😏º🍾🎉¾😞\u2008🏾😅😭👻😥😔😓🏽🎆✓◾🍻🍽🎶🌺🤔😪\x08‑؟🐰🐇🐱🙆．😨⬅🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚地獄谷℅»ВулканПвоАН🐾🐕❣😆ה⋅🔗¿¬🚽歌舞伎🙈😴🏿🤗🇺🇸♫мυтѕＣＭ⤵🏆🎃β😩█▓▒░\u200a🌠🐟💫💰💎⇒эпрд\x95🖐🙅⛲🍰⭐🤐👆›🙌\u2002💛🙁👀🙊🙉¡₂₃\u2004❧▰ˢᵒʳʸ▔ᴼᴷᴺʷᵗʰᵉᵘ◞▀\x13🚬▂▃▄▅▆▇↙🤓\ue602😵άοόςέγὸ̄תמדףנרךצט😒͝″☹➡«🆕👅👥👄🔄🔤👉👤👶👲🔛🎓φ\uf0b7⅓„✋：\uf04c\x9f\x10成都¥😣⏺̲̅😌🤑́🌏😯ех😲∙‛Ἰᾶὁ💞🚓◇🔔📚✏🏀👐\u202d💤🍇\ue613小土豆🏡▷❔❓⁉❗\u202f👠¶》कर्मा🇹🇼🌸蔡英文🌞˚🎲レクサス😛˙外国人关系）Ссиб💋💀🎄💜🤢َِʿьыгя✨不是。ɑ\x80\x9c\x9d🗑\u2005💃📣👿༼つ◕༽😰ḷЗз▱ц￼🤣卖！温哥华议会下降％你失去所有的钱加拿大坏税骗子🐝¯ツ🎅\x85🍺آإشء−ﬂﬁ🎵🌎͟ἔ油别克🤡🤥😬🤧й\u2003₁²🚀🤴ʌʲш¼⁴⁄₄⌠чИОРФДЯМю♭ж✘😝🖑ὐύύ特殊作戦群╪щ💨圆明园ק▶ℐ☭✭🏈😺♪🌍⏏ệ🍔🐮🍁☔🍆🍑🌮🌯☠🤦\u200d♂𝓒𝓲𝓿𝓵안영하세요ЖљКћ🍀😫🤤ῦ我出生在了可以说普通话汉语好极🎼🕺☃🍸🥂🗽🎇🎊🆘☎🤠👩✈🖒✌✰❆☙🚪天一家⚲\u2006⚭⚆⬭⬯⏖○‣⚓新年∎ℒ▪▙☏⅛✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴看ʁ𝗪𝗵𝗲𝗻𝘆𝗼

In [18]:
symbols_to_delete = ''.join([c for c in jigsaw_symbols if not c in glove_symbols])
symbols_to_delete

'\n🍕\r🐵😑\xa0\ue014\t\uf818\uf04a\xadɒ😢🐶️ɡ\uf0e0😜😎👊\u200b\u200e😁😍💖💵ЕＯ👎😀😂\u202a\u202c🔥😄🏻💥ᴅᴋʜᴜᴄᴘʙᴡ😋👏😱‼\x81\u2009🚌🌟😊😳😧🙀😐😕\u200f👍😮😃😘💩💯🚄🏼😖ᴠ🚲‐😟😈ˌ💪🙏🎯🌹😇💔😡\x7f👌ἐὶὲἀῃἴ🙄ＳＵＰＥＲＨＩＴ😠\ufeff\u2028😉😤🙂\u3000👮💙😏🍾🎉😞\u2008🏾😅😭👻😥😔😓🏽🎆🍻🍽🎶🌺🤔😪\x08‑🐰🐇🐱🙆😨🙃💕𝘊𝘦𝘳𝘢𝘵𝘰𝘤𝘺𝘴𝘪𝘧𝘮𝘣💗💚ВПАН🐾🐕😆🔗🚽伎🙈😴🏿🤗🇺🇸ＣＭ🏆🎃😩\u200a🌠🐟💫💰💎\x95🖐🙅🍰🤐👆🙌\u2002💛🙁👀🙊🙉\u2004ˢʳʸᴼᴷᴺᵗʰ\x13🚬🤓\ue602😵ף😒🆕👅👥👄🔄🔤👉👤👶👲🔛🎓\uf0b7\uf04c\x9f\x10😣⏺😌🤑🌏😯😲Ἰᾶὁ💞🚓🔔📚🏀👐\u202d💤🍇\ue613🏡⁉\u202f👠》ा🇹🇼🌸蔡🌞🎲😛关С💋💀🎄💜🤢\x9c\x9d🗑\u2005💃📣👿༼༽😰ḷЗ￼🤣卖华议钱拿坏骗🐝🎅\x85🍺ﬂﬁ🎵🌎ἔ别🤡🤥😬🤧\u2003🚀🤴ʲИОРФДЯМ😝🖑ὐύ殊💨圆园🏈😺🌍⏏ệ🍔🐮🍁🍆🍑🌮🌯🤦\u200d𝓒𝓲𝓿𝓵ЖљК🍀😫🤤ῦ汉语极🎼🕺🍸🥂🗽🎇🎊🆘🤠👩🖒🚪\u2006⬭⬯⏖✀╌🇫🇷🇩🇪🇮🇬🇧😷🇨🇦ХШ🌐\x1f杀鸡给猴ʁ𝗪𝗵𝗲𝗻𝘆𝗼𝘂𝗿𝗮𝗹𝗶𝘇𝗯𝘁𝗰𝘀𝘅𝗽𝘄𝗱📺\u2000սᴦᎥһ\u2007հ\u2001൦ƽ𝐓𝐡𝐞𝐫𝐮𝐝𝐚𝐃𝐜𝐩𝐭𝐢𝐨𝐧ƄᴨᑯΤᏧ௦Іᴑ܁𝐬𝐰𝐲𝐛𝐦𝐯𝐑𝐙𝐣𝐇𝐂𝐘𝟎ԜТᗞ౦〔Ꭻ𝐳𝐔𝐱𝟔𝟓𝐅🐋ﬃ💘💓𝘥𝘯𝘶💐🌋🌄🌅𝙬𝙖𝙨𝙤𝙣𝙡𝙮𝙘𝙠𝙚𝙙𝙜𝙧𝙥𝙩𝙪𝙗𝙞𝙝𝙛👺🐷ℋ𝐀𝐥𝐪🚶𝙢Ἱ🤘💸Ｗ𝙇ʒᵻ👂👃🎫\uf0a7БУ🚢🚂ગજરાતῆ🏃𝓬𝓻𝓴𝓮𝓽𝓼﴾﴿Φ₽\ue807𝑻𝒆𝒍𝒕𝒉𝒓𝒖𝒂𝒏𝒅𝒔𝒎𝒗𝒊👽😙\u200cЛ‒🎾👹⎌🏒⛸寓养宠🏄🐀🚑🤷𝒑𝒚𝒐𝑴🤙🐒欢拉斯𝙫🐈𝒌𝙊𝙭𝙆𝙋𝙍𝘼𝙅🦄收赢愤买ẽ🚗🐳𝟏𝐟𝟖𝟑𝟕𝒄𝟗𝐠𝙄𝙃👇锟拷𝗢𝟳𝟱𝟬⛷𝘿𝙔₵𝒩𝒾𝓁𝒶𝓉𝓇𝓊𝓃𝓈𝓅𝒻𝒽𝓀𝓌𝒸𝓎𝙏𝙟𝘃𝗺𝟮𝟭𝟯𝟲👋🦊伦ǫ🐽🎻🎹⛓🏹🍷🦆为谊贺其猜传积认识督曾经让稣复们聊些题战胜圣结孩惧谓样还🎸🤕🤒⛑🎁批检讨🏝🦁Δ🙋😶뤼렵🔫👁ὰ💲🗯𝙈Ἄ𝒇𝒈𝒘𝒃𝑬𝑶𝕾𝖙𝖗𝖆𝖎𝖌𝖍𝖕𝖊𝖔𝖑𝖉𝖓𝖐𝖜𝖞𝖚𝖇𝕿𝖘𝖄𝖛𝖒𝖋𝖂𝕴𝖟𝖈𝕸👑🚿💡\uf005𝙀𝒛𝑲𝑳𝑾𝒋𝟒😦𝙒𝘾𝘽🏐𝘩𝘨ὼṑ𝑱𝑹𝑫𝑵𝑪🇰🇵👾ᓇᒧᔭᐃᐧᐦᑳᐨᓃᓂᑲᐸᑭᑎᓀᐣ🐄🎈🔨🐎🤞🐸💟🎰🌝🛳

In [19]:
symbols_to_isolate = ''.join([c for c in jigsaw_symbols if c in glove_symbols])
symbols_to_isolate

'.,?!-;*"…:—()%#$&_/@＼・ω+=”“[]^–>\\°<~•≠™ˈʊ∞§{}·τα❤☺|عدويهصقأناخلىبمغر¢→̶`❥━┣┫┗►★©―ᴍʏʀɪᴇɴᴏᴀʟᴛғᴊɢ✔®\x96\x92●שלוםבי£♥エンジ故障➤´ᴵ͞¹☕≈÷אעכח♡◐║▬′ɔː⛽€ஜ۩۞†μ✒➥═☆◄½ʻπδηλήσεικίρξνʃ✬☻±⛺♍µتحكسةفزطº¾✓◾؟．⬅地獄谷℅»улканво❣ה⋅¿¬歌舞♫мυтѕ⤵β█▓▒░⇒эпрд⛲⭐›¡₂₃❧▰ᵒ▔ʷᵉᵘ◞▀▂▃▄▅▆▇↙άοόςέγὸ̄תמדנרךצט͝″☹➡«φ⅓„✋：成都¥̲̅́ех∙‛◇✏小土豆▷❔❓❗¶कर्म英文˚レクサス˙外国人系）сибَِʿьыгя✨不是。ɑ\x80つ◕з▱ц！温哥会下降％你失去所有的加大税子¯ツآإشء−͟油克й₁²ʌш¼⁴⁄₄⌠чю♭ж✘ύ特作戦群╪щ明ק▶ℐ☭✭♪☔☠♂안영하세요ћ我出生在了可以说普通话好☃☎✈✌✰❆☙天一家⚲⚭⚆○‣⚓新年∎ℒ▪▙☏⅛看ｃϖүａͺｓǀɩ℮ｙｅｌ¸ｗｈ‚ן໐∼ё‖ℳ❄←ͦ☼ج패티⋆ɜі⊂ુી、⅔☘¨͡๏̯⚾⚽×θ￦公物吗？（操美℃迎来到阿ספ⏩☮ﷻ⚠巨得月白鬼怒要额✊斤❌⭕▸⦁マルハニチロ株式社한국어ㄸㅓ니͜ʖ■⇌ℯℴζ☐☑多⚡☄╭∩╮和中友祝与想象对法如直接问，用自己本教士没唯基徒相信耶活死怪他但当政治时候例因把全堂婚恐且栗这♾判＞ʕɐ̣₀쥐스탱트도석유가격인상이경제황을게만들지않록잘관리해야합다캐나에서대마초와화약금의품런성분갈때는반드시허된사용✞┈╱╲▏▕┃╰▊▋╯┳┊≥☒凸↑☝知彼百ɹ✅☛♩☞点版っ◔◡↓ф♀⬆چی\x91⠀╚⏰↺∏鉄リ事件✾◦♬³ї製シの虚偽屁理｜／∵∴√χ¤ψաիձ冬至ὀک☜ώ▲♲↴↳▫‿⬇➕✧ｇｏｖｒｎｍｔｉｄｕ－２０８ｆｂ＇ｋ‰≤∕ˆ⚜☁ლڡさようなら'

In [0]:
isolate_dict = {ord(c):f' {c} ' for c in symbols_to_isolate}
remove_dict = {ord(c):f'' for c in symbols_to_delete}


def handle_punctuation(x):
    x = x.translate(remove_dict)
    x = x.translate(isolate_dict)
    return x

In [21]:
train['comment_text'] = train['comment_text'].progress_apply(lambda x:handle_punctuation(x))
test['comment_text'] = test['comment_text'].progress_apply(lambda x:handle_punctuation(x))

100%|██████████| 97320/97320 [00:03<00:00, 29380.19it/s]


In [22]:
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())))
oov = check_coverage(vocab,glove_embeddings)
oov[:10]

100%|██████████| 543276/543276 [00:00<00:00, 581238.19it/s]


Found embeddings for 14.32% of vocab
Found embeddings for  85.95% of all text


[('I', 892214),
 ('The', 454818),
 ('Trump', 197670),
 ("don't", 185012),
 ('It', 157783),
 ('You', 148849),
 ('If', 148520),
 ('And', 137284),
 ('This', 124707),
 ("it's", 104143)]

In [0]:
from nltk.tokenize.treebank import TreebankWordTokenizer
tokenizer = TreebankWordTokenizer()

def handle_contractions(x):
    x = tokenizer.tokenize(x)
    x = ' '.join(x)
    return x

In [27]:
train['comment_text'] = train['comment_text'].progress_apply(lambda x:handle_contractions(x))
test['comment_text'] = test['comment_text'].progress_apply(lambda x:handle_contractions(x))

100%|██████████| 97320/97320 [00:22<00:00, 4357.64it/s]


In [28]:
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embeddings)
oov[:10]

100%|██████████| 492597/492597 [00:00<00:00, 561620.42it/s]


Found embeddings for 15.85% of vocab
Found embeddings for  87.17% of all text


[('I', 1044087),
 ('The', 454914),
 ('It', 245716),
 ('Trump', 224602),
 ('You', 169099),
 ('If', 148538),
 ('And', 137290),
 ('This', 124742),
 ('That', 113153),
 ('They', 112173)]

In [0]:
## If you need to fix qouote 

import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x


In [35]:
# train['comment_text'][1777:1785]

train['comment_text'] = train['comment_text'].progress_apply(lambda x:clean_numbers(x))
test['comment_text'] = test['comment_text'].progress_apply(lambda x:clean_numbers(x))

100%|██████████| 97320/97320 [00:03<00:00, 31405.10it/s]


In [36]:
vocab = build_vocab(list(train['comment_text'].apply(lambda x:x.split())),verbose=False)
oov = check_coverage(vocab,glove_embeddings)
oov[:10]

100%|██████████| 476007/476007 [00:00<00:00, 539089.38it/s]


Found embeddings for 16.40% of vocab
Found embeddings for  87.74% of all text


[('I', 1044087),
 ('The', 454914),
 ('It', 245716),
 ('Trump', 224602),
 ('You', 169099),
 ('If', 148538),
 ('And', 137290),
 ('This', 124742),
 ('That', 113153),
 ('They', 112173)]

In [0]:
# def clean_text(x):

#     x = str(x)
#     for punct in "/-'":
#         x = x.replace(punct, ' ')
#     for punct in '&':
#         x = x.replace(punct, f' {punct} ')
#     for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
#         x = x.replace(punct, '')
#     return x
# train["question_text"] = train["question_text"].progress_apply(lambda x: clean_text(x))
# sentences = train["question_text"].apply(lambda x: x.split())
# vocab = build_vocab(sentences)